In [40]:
import pandas as pd
import pickle
from glob import glob

def _add_correct(data, thresh=2.0):
    data = data.copy()
    for col in data:
        if 'rmsd' in col:
            data[col.replace('rmsd', 'correct')] = round(data[col], 1) <= thresh
    return data

def load(root, version, helpers, mcss=None):
    print('{}/*/scores/{}/summary/{}.tsv'.format(root, version, helpers))
    fnames = glob('{}/*/scores/{}/summary/{}.tsv'.format(root, version, helpers))
    data = pd.concat(pd.read_csv(fname, sep='\t')
                     for fname in fnames)
    data['version'] = version
    data['helpers'] = helpers
    data = _add_correct(data)
    if mcss is not None:
        with open(mcss, 'rb') as fp:
            mcss = pd.DataFrame(pickle.load(fp).items(), columns=['ligand', 'mcss']).set_index('ligand')
            data = data.join(mcss, on='ligand')
    return data

data1 = load('/Users/jpaggi/sherlock/oak/users/jpaggi/ionchannels', 'stats104', 'best_affinity_diverse',
             '/Users/jpaggi/sherlock/oak/users/jpaggi/ionchannels_mcss.pkl')
data2 = load('/Users/jpaggi/sherlock/oak/users/jpaggi/bpp_data', 'stats104', 'best_affinity_diverse',
             '/Users/jpaggi/sherlock/oak/users/jpaggi/stats102_mcss_sizes.pkl')
data = pd.concat([data1, data2])
data = data[~data['protein'].isin(['O35433', 'P13569', 'O14649'])]

/Users/jpaggi/sherlock/oak/users/jpaggi/ionchannels/*/scores/stats104/summary/best_affinity_diverse.tsv
/Users/jpaggi/sherlock/oak/users/jpaggi/bpp_data/*/scores/stats104/summary/best_affinity_diverse.tsv


In [41]:
len(set(data['protein'])), len(data)

(31, 423)

In [39]:
data[data.best_correct].filter(regex="correct").mean()

combind_correct    0.740299
glide_correct      0.635821
best_correct       1.000000
dtype: float64

In [35]:
data[data.best_correct & (data.mcss < 0.5)].filter(regex="correct").mean()

combind_correct    0.692308
glide_correct      0.576923
best_correct       1.000000
dtype: float64

In [36]:
data[data.best_correct].groupby('protein').sum().filter(regex="correct")

,combind_correct,glide_correct,best_correct
protein,,,
5HT2B,3.0,2.0,5.0
AR,17.0,16.0,17.0
B1AR,9.0,3.0,11.0
B2AR,6.0,5.0,7.0
BACE1,2.0,2.0,7.0
BRD4,5.0,1.0,10.0
CDK2,11.0,11.0,17.0
DAT,5.0,3.0,7.0
DHFR,9.0,8.0,15.0


In [20]:
data1.groupby('protein').sum().filter(regex="correct")

,combind_correct,glide_correct,best_correct
protein,,,
O14649,0.0,0.0,1.0
O35433,1.0,1.0,1.0
P13569,0.0,0.0,1.0
P19491,10.0,8.0,12.0
P22756,8.0,7.0,15.0
Q05586-Q12879,6.0,6.0,6.0


In [9]:
data[data.mcss <= 0.5].filter(regex="correct").sum()

combind_correct    13
glide_correct       8
best_correct       18
dtype: int64